In [0]:
!pip install nba-api
!pip install Keras

     |████████████████████████████████| 225kB 9.7MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# dependencies

# basic
from datetime import datetime
import json
import pandas as pd
from pandas import ExcelFile
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
import requests
import seaborn as sns
import time

# nba api
import nba_api
from nba_api.stats.endpoints import BoxScoreDefensive, BoxScoreMiscV2, BoxScorePlayerTrackV2, BoxScoreUsageV2
from nba_api.stats.static import teams
from nba_api.stats.endpoints import LeagueGameFinder, LeagueGameLog, PlayerGameLog

# sklearn
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, Normalizer, StandardScaler
from sklearn.svm import SVC

# tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

# csv

In [0]:
df_final = pd.read_csv('/content/drive/My Drive/mo_bucks_ml/current/gamestatsfinal_version_3.csv')
# df_final = pd.read_csv('gamestatsfinal_version_3.csv')

In [0]:
df_final

,AST_PCT,AST_RATIO,AST_TOV,AST,BLK,BLKA,BLK.1,CFGA,CFGM,CFG_PCT,COMMENT,Capacity,Capacity_pct,City,DEF_RATING,DFGA,DFGM,DFG_PCT,DIST,DRBC,DREB_PCT,DREB,EFG_EVAL,EFG_PCT,EFG_TEAM_AVG,E_DEF_RATING,E_NET_RATING,E_OFF_RATING,E_PACE,E_USG_PCT,FG3A,FG3M,FG3_PCT,FGA,FGM,FG_PCT,FTA,FTAST,FTM,FT_PCT,...,PLAYER_NAME,PLAYER_PTS,PLUS_MINUS,POSS,PTS,PTS_2ND_CHANCE,PTS_FB,PTS_OFF_TOV,PTS_PAINT,RBC,REB,REB_PCT,SAST,SEASON_ID,SPD,START_POSITION,STL,SWITCHES_ON,TCHS,TEAM_ABBREVIATION,TEAM_CITY,TEAM_ID,TEAM_NICKNAME,TM_TOV_PCT,TOV,TRUE_PERFORMANCE,TRUE_PERFORMANCE_EVAL,TRUE_PERFORMANCE_TEAM_AVG,TS_EVAL,TS_PCT,TS_TEAM_AVG,UFGA,UFGM,UFG_PCT,USG_PCT,VIDEO_AVAILABLE,WL,YEAR BUILT,ARENA,ATTENDANCE
0,0.214,27.3,3.00,3,0,1,0,2,0,0.000,NaN,18624,1.000000,Boston Massachusetts,81.0,4,3,0.750,1.37,3,0.050,1,0.100583,0.571,0.470417,82.8,22.9,105.6,100.87,0.175,4,2,0.500,7,3,0.429,0,0,0,0.0,...,Aron Baynes,16,8,42,8,0,0,0,2,8,4,0.095,0,22018,3.93,NaN,0,0,30,BOS,Boston,1610612738,Celtics,9.1,1,0.225529,0.048583,0.176946,0.090667,0.571,0.480333,5,3,0.600,0.163,1,W,1995,TD Garden,18624
1,0.091,11.1,1.00,2,0,2,0,6,2,0.333,NaN,18624,1.000000,Boston Massachusetts,92.2,0,0,0.000,2.28,6,0.063,2,-0.047417,0.423,0.470417,89.6,17.4,107.0,110.97,0.227,3,1,0.333,13,5,0.385,2,0,1,0.5,...,Jaylen Brown,4,11,64,12,2,0,2,6,11,5,0.078,1,22018,4.50,G,0,0,30,BOS,Boston,1610612738,Celtics,11.1,2,0.205882,0.028936,0.176946,-0.048333,0.432,0.480333,7,3,0.429,0.229,1,W,1995,TD Garden,18624
2,0.000,0.0,0.00,0,0,1,0,5,1,0.200,NaN,18624,1.000000,Boston Massachusetts,77.8,1,0,0.000,1.81,8,0.097,3,-0.095417,0.375,0.470417,76.6,19.6,96.2,106.19,0.209,3,1,0.333,12,4,0.333,2,0,1,0.5,...,Gordon Hayward,3,10,54,10,3,0,2,0,12,5,0.081,1,22018,4.07,F,4,0,38,BOS,Boston,1610612738,Celtics,0.0,0,0.218059,0.041113,0.176946,-0.092333,0.388,0.480333,7,3,0.429,0.206,1,W,1995,TD Garden,18624
3,0.095,16.7,0.67,2,4,0,4,3,3,1.000,NaN,18624,1.000000,Boston Massachusetts,85.3,11,6,0.545,2.15,12,0.103,4,0.100583,0.571,0.470417,83.2,13.5,96.7,111.39,0.136,1,0,0.000,7,4,0.571,1,1,1,1.0,...,Al Horford,17,9,68,9,0,0,0,6,12,4,0.056,0,22018,3.99,C,0,0,52,BOS,Boston,1610612738,Celtics,25.0,3,0.150706,-0.026240,0.176946,0.124667,0.605,0.480333,4,1,0.250,0.137,1,W,1995,TD Garden,18624
4,0.292,28.0,2.33,7,0,1,0,2,1,0.500,NaN,18624,1.000000,Boston Massachusetts,89.4,1,1,1.000,2.25,7,0.118,4,-0.291417,0.179,0.470417,87.3,12.1,99.4,112.55,0.245,8,1,0.125,14,2,0.143,2,0,2,1.0,...,Kyrie Irving,7,9,66,7,0,0,0,2,9,4,0.059,1,22018,4.33,G,0,0,81,BOS,Boston,1610612738,Celtics,12.0,3,0.163000,-0.013946,0.176946,-0.245333,0.235,0.480333,12,1,0.083,0.247,1,W,1995,TD Garden,18624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22457,0.600,45.0,9.00,9,0,0,0,4,4,1.000,NaN,19441,1.019186,Portland Oregon,122.9,0,0,0.000,1.20,1,0.067,1,0.172909,0.850,0.677091,120.7,59.0,179.6,100.73,0.293,1,1,1.000,10,8,0.800,0,0,0,0.0,...,De'Aaron Fox,12,19,35,17,3,0,5,14,2,1,0.042,0,22018,4.08,G,0,0,36,SAC,Sacramento,1610612758,Kings,5.0,1,0.165765,-0.031904,0.197668,0.161182,0.850,0.688818,6,4,0.667,0.282,1,L,1995,Moda Center,19814
22458,0.105,22.2,2.00,2,0,1,0,1,0,0.000,NaN,19441,1.019186,Portland Oregon,120.6,2,1,0.500,1.41,4,0.143,2,0.072909,0.750,0.677091,118.4,56.3,174.6,100.19,0.196,5,3,0.600,6,3,0.500,0,0,0,0.0,...,Buddy Hield,6,16,33,9,0,3,3,0,6,3,0.130,0,22018,4.99,G,0,0,22,SAC,Sacramento,1610612758,Kings,11.1,1,0.210471,0.012802,0.197668,0.061182,0.750,0.688818,5,3,0.600,0.189,1,L,1995,Moda Center,19814
22459,0.000,0.0,0.00,0,0,0,0,0,0,0.000,NaN,19441,1.019186,Portland Oregon,123.1,1,1,1.000,0.44,0,0.000,0,-0.177091,0.500,0.677091,134.7,-26.0,108.7,98.22,0.155,1,0,0.000,2,1,0.500,0,0,0,0.0,...,BJ Johnson,6,-2,12,2,0,0,2,0,0,0,0.000,0,22018,4.00,NaN,0,0,5,SAC,Sacrame

In [0]:
# fix gameid column w/ leading zeros
df_final['GAME_ID'] = df_final['GAME_ID'].map(lambda x: f'{x:0>10}')

In [0]:
df_final.head()

,AST_PCT,AST_RATIO,AST_TOV,AST,BLK,BLKA,BLK.1,CFGA,CFGM,CFG_PCT,COMMENT,Capacity,Capacity_pct,City,DEF_RATING,DFGA,DFGM,DFG_PCT,DIST,DRBC,DREB_PCT,DREB,EFG_EVAL,EFG_PCT,EFG_TEAM_AVG,E_DEF_RATING,E_NET_RATING,E_OFF_RATING,E_PACE,E_USG_PCT,FG3A,FG3M,FG3_PCT,FGA,FGM,FG_PCT,FTA,FTAST,FTM,FT_PCT,...,PLAYER_NAME,PLAYER_PTS,PLUS_MINUS,POSS,PTS,PTS_2ND_CHANCE,PTS_FB,PTS_OFF_TOV,PTS_PAINT,RBC,REB,REB_PCT,SAST,SEASON_ID,SPD,START_POSITION,STL,SWITCHES_ON,TCHS,TEAM_ABBREVIATION,TEAM_CITY,TEAM_ID,TEAM_NICKNAME,TM_TOV_PCT,TOV,TRUE_PERFORMANCE,TRUE_PERFORMANCE_EVAL,TRUE_PERFORMANCE_TEAM_AVG,TS_EVAL,TS_PCT,TS_TEAM_AVG,UFGA,UFGM,UFG_PCT,USG_PCT,VIDEO_AVAILABLE,WL,YEAR BUILT,ARENA,ATTENDANCE
0,0.214,27.3,3.00,3,0,1,0,2,0,0.000,NaN,18624,1.0,Boston Massachusetts,81.0,4,3,0.750,1.37,3,0.050,1,0.100583,0.571,0.470417,82.8,22.9,105.6,100.87,0.175,4,2,0.500,7,3,0.429,0,0,0,0.0,...,Aron Baynes,16,8,42,8,0,0,0,2,8,4,0.095,0,22018,3.93,NaN,0,0,30,BOS,Boston,1610612738,Celtics,9.1,1,0.225529,0.048583,0.176946,0.090667,0.571,0.480333,5,3,0.600,0.163,1,W,1995,TD Garden,18624
1,0.091,11.1,1.00,2,0,2,0,6,2,0.333,NaN,18624,1.0,Boston Massachusetts,92.2,0,0,0.000,2.28,6,0.063,2,-0.047417,0.423,0.470417,89.6,17.4,107.0,110.97,0.227,3,1,0.333,13,5,0.385,2,0,1,0.5,...,Jaylen Brown,4,11,64,12,2,0,2,6,11,5,0.078,1,22018,4.50,G,0,0,30,BOS,Boston,1610612738,Celtics,11.1,2,0.205882,0.028936,0.176946,-0.048333,0.432,0.480333,7,3,0.429,0.229,1,W,1995,TD Garden,18624
2,0.000,0.0,0.00,0,0,1,0,5,1,0.200,NaN,18624,1.0,Boston Massachusetts,77.8,1,0,0.000,1.81,8,0.097,3,-0.095417,0.375,0.470417,76.6,19.6,96.2,106.19,0.209,3,1,0.333,12,4,0.333,2,0,1,0.5,...,Gordon Hayward,3,10,54,10,3,0,2,0,12,5,0.081,1,22018,4.07,F,4,0,38,BOS,Boston,1610612738,Celtics,0.0,0,0.218059,0.041113,0.176946,-0.092333,0.388,0.480333,7,3,0.429,0.206,1,W,1995,TD Garden,18624
3,0.095,16.7,0.67,2,4,0,4,3,3,1.000,NaN,18624,1.0,Boston Massachusetts,85.3,11,6,0.545,2.15,12,0.103,4,0.100583,0.571,0.470417,83.2,13.5,96.7,111.39,0.136,1,0,0.000,7,4,0.571,1,1,1,1.0,...,Al Horford,17,9,68,9,0,0,0,6,12,4,0.056,0,22018,3.99,C,0,0,52,BOS,Boston,1610612738,Celtics,25.0,3,0.150706,-0.026240,0.176946,0.124667,0.605,0.480333,4,1,0.250,0.137,1,W,1995,TD Garden,18624
4,0.292,28.0,2.33,7,0,1,0,2,1,0.500,NaN,18624,1.0,Boston Massachusetts,89.4,1,1,1.000,2.25,7,0.118,4,-0.291417,0.179,0.470417,87.3,12.1,99.4,112.55,0.245,8,1,0.125,14,2,0.143,2,0,2,1.0,...,Kyrie Irving,7,9,66,7,0,0,0,2,9,4,0.059,1,22018,4.33,G,0,0,81,BOS,Boston,1610612738,Celtics,12.0,3,0.163000,-0.013946,0.176946,-0.245333,0.235,0.480333,12,1,0.083,0.247,1,W,1995,TD Garden,18624


# boxscores concat

In [0]:
test_array = df_final['GAME_ID'].unique()[:10]
unique_array = df_final['GAME_ID'].unique()
test_df = df_final.copy()

In [0]:
missing_games = [
'0021801226',
'0021801218'
]

In [0]:
df_def_list = []
df_misc_list = []
df_tracking_list = []
df_usage_list = []

startTime = datetime.now()

game_ok_count = 0
game_no_count = 0

game_problem_list = []

for game in missing_games:
  
    try:

        print(f'Searching for game: {game}...')

        df_defensive = BoxScoreDefensive(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 1.Defensive df found and saved')

        df_misc = BoxScoreMiscV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 2.Misc df found and saved')

        df_tracking = BoxScorePlayerTrackV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 3.Tracking df found and saved')

        df_usage = BoxScoreUsageV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 4.Usage df found and saved')

#         x, = np.where(unique_array == game)
        x, = np.where(missing_games == game)

        game_ok_count += 1

        print(f'--- {x+1} games searched ---')
        print(f'--- {game_ok_count} games found ---')

        time.sleep(5)

        df_def_list.append(df_defensive)
        df_misc_list.append(df_misc)
        df_tracking_list.append(df_tracking)
        df_usage_list.append(df_usage)

        print(f'--- {game_ok_count} games appended ---')

        print(f'--- Time: {datetime.now() - startTime} ---')

    except:
    
    #   problem list
        game_no_count += 1

        game_problem_list.append(game)

        print(f'game {game} encountered timeout')
        print(f'--- {game_no_count} games with problems ---')

    #   sleep
        print(f'--- sleeping for 300 seconds ---')
        time.sleep(300)

    #   try again to search
        try:

            print(f'Searching again for game: {game}...')

            df_defensive = BoxScoreDefensive(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 1.Defensive df found and saved')

            df_misc = BoxScoreMiscV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 2.Misc df found and saved')

            df_tracking = BoxScorePlayerTrackV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 3.Tracking df found and saved')

            df_usage = BoxScoreUsageV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 4.Usage df found and saved')

            x, = np.where(unique_array == game)

            game_ok_count += 1

            print(f'--- {x+1} games searched ---')
            print(f'--- {game_ok_count} games found ---')

            time.sleep(5)

            df_def_list.append(df_defensive)
            df_misc_list.append(df_misc)
            df_tracking_list.append(df_tracking)
            df_usage_list.append(df_usage)

            print(f'--- {game_ok_count} games appended ---')

            print(f'--- Time: {datetime.now() - startTime} ---')

        except:

            print(f'game {game} encountered another timeout')
            print(f'--- {game_no_count} games with problems ---')

        #   sleep
            print(f'--- sleeping for 300 seconds ---')
            time.sleep(300)

            continue

    continue

# df_def_all = pd.concat(df_def_list)
# df_misc_all = pd.concat(df_misc_list)
# df_tracking_all = pd.concat(df_tracking_list)
# df_usage_all = pd.concat(df_usage_list)

df_def_missing = pd.concat(df_def_list)
df_misc_missing = pd.concat(df_misc_list)
df_tracking_missing = pd.concat(df_tracking_list)
df_usage_missing = pd.concat(df_usage_list)

print(f'--- merged df ---')

In [0]:
df_def_all.to_csv('defensive.csv')
df_misc_all.to_csv('misc.csv')
df_tracking_all.to_csv('tracking.csv')
df_usage_all.to_csv('usage.csv')

In [0]:
# stats_df = test_df.merge(df_def_all, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')
stats_df = test_df.merge(df_def_missing, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')

In [0]:
# stats_df = test_df.merge(df_misc_all, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')
stats_df = test_df.merge(df_misc_missing, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')

In [0]:
# stats_df = test_df.merge(df_tracking_all, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')
stats_df = test_df.merge(df_tracking_missing, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')

In [0]:
# stats_df = test_df.merge(df_usage_all, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')
stats_df = test_df.merge(df_usage_missing, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')

In [0]:
stats_df